**1. What are Bollinger Bands?**

Bollinger Bands are an indicator in technical chart analysis that uses a simple moving average (SMA) and standard deviation to spot periods of high or low volatility. Bollinger Bands are typically visualized as three lines on top of a price chart. The middle line represents the SMA, whereas the upper and lower lines indicate the SMA +/- one standard deviation. They are named after financial analyst [John Bollinger](https://en.wikipedia.org/wiki/John_Bollinger).

If the upper and lower bands move closer together, they squeeze the SMA in the middle, a sign of low volatility. Periods of low volatility are usually followed by periods of higher price swings which will cause the bands to widen. However, neither the contraction nor the widening of the bands alone can tell you when the change in volatility will occur nor can they predict in which direction the market will move.

Another interesting feature of Bollinger Bands are breakouts. Breakouts are events in which the price of an asset moves below or above one of the outer bands. These breakouts can be an indicator for a price correction. However, they should not be treated as a trading signal.

To deepen our understanding of how to work with Bollinger Bands we will here calculate them for the Bitcoin price and visualize them in an interactive chart. So, to begin with we will first see how we can create an interactive chart for the Bitcoin price and then overlay it with the Bollinger Bands. 

So, let's get started by importing all the Python packages we need and loading the latest Bitcoin data from the [400+ crypto currency pairs at 1-minute resolution dataset.](https://www.kaggle.com/tencars/392-crypto-currency-pairs-at-minute-resolution)


In [ ]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go

# Load the Bitcoin price data
df = pd.read_csv('/kaggle/input/392-crypto-currency-pairs-at-minute-resolution/btcusd.csv')

**2. How to visualize OHLC data**

The easiest way of displaying the Bitcoin data would be to create a line plot of the closing price using [Matplotlib](https://matplotlib.org/) or the [Pandas plotting function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html). However, by doing so we would lose a lot of information. Besides the closing the dataset also contains the opening price as well as the highest and lowest price in each 1-Minute bin. Obviously, it would be good to have this information preserved in the visualization. A common way of displaying all this data within one chart are candlestick plots. The concept of candlestick plots is outlined in the figure below where a green, bullish candle indicates a higher closing price than the opening price. Vice versa a red bearish candle shows a lower closing price than the opening price. The ticks at the top and the bottom of each candle show the highest and the lowest price during the candle time interval, here 1-Minute.

<img src="https://upload.wikimedia.org/wikipedia/commons/7/72/Candle_Definition.png" width="800px">

This way all the information in our dataset can be visualized. Of course we could now write our own plotting function to create such a candlestick chart, but there is an out *of the box alternative* already available: [Plotly](https://plot.ly/)

Plotly already contains the functionality to create candlestick plots, plus the charts are interactive. So, let's have a look! The code below is all we need to get our first candlestick chart running.

In [ ]:
# Create a working copy of the last 2 hours of the data
btc_data = df.iloc[-120:].copy()

# Create an interactive candlestick plot with Plotly
fig = go.Figure(data=[go.Candlestick(x=pd.to_datetime(btc_data.time, unit='ms'),
                open=btc_data['open'],
                high=btc_data['high'],
                low=btc_data['low'],
                close=btc_data['close'])])

# Add title and format axes
fig.update_layout(
    title='Bitcoin price in the last 2 hours as candlestick chart',
    yaxis_title='BTC/USD')

fig.show()

This looks good! The last two hours of the Bitcoin price from our dataset are shown in the candlestick chart. By moving the cursor into the plot we get all the relevant data shown in a small window and we can zoom into parts of the chart that are of particular interest for us.

**3. Adding the Bollinger Bands to the chart**

Now let's see what else we can do. After all the idea here is to create the Bollinger Bands to get an indicator for the volatility in the data. 

As discussed previously the central line of the Bollinger Bands indicator is a SMA. This can be easily calculated with the Pandas toolbox by using the *rolling* method combined with *mean*. Typically, Bollinger Bands are calculated on daily charts with a time window of 20 days. But here we have minute resolution data so instead of using a 20 days window we will use a 20 minutes window. Next, we need to calculate the standard deviation in the same fashion. The resulting standard deviation will then be multiplied by two and added/subtracted from the SMA to create the upper/lower Bollinger Bands.

Finally, the result can be visualized by plotting the Bollinger Bands on top of our previous chart.

In [ ]:
# Create a working copy of the last 2 hours of the data
btc_data = df.iloc[-240:].copy()

# Define the parameters for the Bollinger Band calculation
ma_size = 20
bol_size = 2

# Convert the timestamp data to a human readable format
btc_data.index = pd.to_datetime(btc_data.time, unit='ms')

# Calculate the SMA
btc_data.insert(0, 'moving_average', btc_data['close'].rolling(ma_size).mean())

# Calculate the upper and lower Bollinger Bands
btc_data.insert(0, 'bol_upper', btc_data['moving_average'] + btc_data['close'].rolling(ma_size).std() * bol_size)
btc_data.insert(0, 'bol_lower', btc_data['moving_average'] - btc_data['close'].rolling(ma_size).std() * bol_size)

# Remove the NaNs -> consequence of using a non-centered moving average
btc_data.dropna(inplace=True)

# Create an interactive candlestick plot with Plotly
fig = go.Figure(data=[go.Candlestick(x = btc_data.index,
                                     open = btc_data['open'],
                                     high = btc_data['high'],
                                     low = btc_data['low'],
                                     showlegend = False,
                                     close = btc_data['close'])])

# Plot the three lines of the Bollinger Bands indicator
for parameter in ['moving_average', 'bol_lower', 'bol_upper']:
    fig.add_trace(go.Scatter(
        x = btc_data.index,
        y = btc_data[parameter],
        showlegend = False,
        line_color = 'gray',
        mode='lines',
        line={'dash': 'dash'},
        marker_line_width=2, 
        marker_size=10,
        opacity = 0.8))
    
# Add title and format axes
fig.update_layout(
    title='Bitcoin price in the last 2 hours with Bollinger Bands',
    yaxis_title='BTC/USD')

fig.show()

**4. Conclusion**

We have seen here how to calculate and visualize Bollinger Bands, a simple but powerful indicator for technical chart analysis. From the above chart we can see that the Bitcoin price for the time period of interest moves largely within the upper and lower Bollinger Bands. However, occasionally the price breaks out of the bands and then moves back into the area between the two bands. It is also visible from this short time period that breakouts are not a reliable measure of price changes. In some cases, the price moves into another direction after a breakout, whereas in other cases it just moves sideways.

From the implementation point of view we saw that calculating and visualizing the Bollinger Bands is possible with just a view lines of code, thanks to the Python packages [Pandas](https://pandas.pydata.org/) and [Plotly](https://plotly.com/).